In [3]:
from pathlib import Path
import sys

from google.colab import drive
drive.mount('/content/drive')

base_folder = Path('/content/drive/MyDrive/')
data_folder = Path('/content')

!pip install pytorch-lightning==2.0.9 -qq

!pip install torchmetrics -U -qq
!pip install fastdownload -U -qq
!pip install fastai -U -qq
!pip install wandb -U -qq

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install accelerate --upgrade


In [5]:
!pip install transformers[torch]

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import numpy as np
import torch


In [7]:
output_dir = '/content/drive/MyDrive/FoodBuddy/Model2'
test_file = "/content/drive/MyDrive/FoodBuddy/unsupervised_test.txt"
def calculate_perplexity(logits):
    return np.exp(logits.mean())

def calculate_burstiness(text):
    words = text.split()
    word_count = len(words)
    unique_words = len(set(words))
    burstiness = unique_words / word_count
    return burstiness

In [ ]:
def fine_tune_gpt2(model_name, train_file,val_file, output_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Adjust block_size for training and evaluation datasets
    block_size = 64  # Experiment with smaller values for faster processing
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=block_size
    )
    eval_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=val_file,
        block_size=block_size
    )

    # Use smaller batch sizes for faster training
    batch_size = 4  # Experiment with smaller values for faster processing

    # Data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=5,
        save_steps=10000,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    # Trainer setup
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset
    )

    # Training
    trainer.train()

    # Evaluate perplexity
    eval_dataloader = trainer.get_eval_dataloader(eval_dataset)
    model.eval()
    perplexity = []
    for batch in eval_dataloader:
        inputs, labels = batch["input_ids"], batch["labels"]
        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            logits = outputs.logits
            perplexity.append(calculate_perplexity(logits))

    avg_perplexity = np.mean(perplexity)
    print(f"Average Perplexity: {avg_perplexity}")

    # Generate text for burstiness calculation
    generated_text = model.generate(
        max_length=500,  # Adjust length for faster generation
        temperature=0.7,
        top_k=100,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1
    )[0]

    generated_text = tokenizer.decode(generated_text, skip_special_tokens=True)
    burstiness = calculate_burstiness(generated_text)
    print(f"Burstiness: {burstiness}")

    # Save the fine-tuned model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [8]:
def fine_tune_gpt2(model_name, train_file,val_file, output_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Adjust block_size for training and evaluation datasets
    block_size = 64  # Experiment with smaller values for faster processing
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=block_size
    )
    eval_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=val_file,
        block_size=block_size
    )

    # Use smaller batch sizes for faster training
    batch_size = 4  # Experiment with smaller values for faster processing

    # Data collator for language modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Training arguments
    training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=5,
    save_steps=10000,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=5e-6, # Initial learning rate
    max_grad_norm=1.0, # Gradient clipping
    weight_decay=0.04, # Weight decay
    logging_dir=None, # Directory for storing logs
    logging_steps=100, # Interval for logging
    max_steps=500, # Total number of steps for learning rate finder
    find_unused_parameters=True, # Search for unused parameters
    )

    # Trainer setup
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset
    )

    # Training
    trainer.train()

    # Evaluate perplexity
    eval_dataloader = trainer.get_eval_dataloader(eval_dataset)
    model.eval()
    perplexity = []
    for batch in eval_dataloader:
        inputs, labels = batch["input_ids"], batch["labels"]
        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            logits = outputs.logits
            perplexity.append(calculate_perplexity(logits))

    avg_perplexity = np.mean(perplexity)
    print(f"Average Perplexity: {avg_perplexity}")

    # Generate text for burstiness calculation
    generated_text = model.generate(
        max_length=500,  # Adjust length for faster generation
        temperature=0.7,
        top_k=100,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1
    )[0]

    generated_text = tokenizer.decode(generated_text, skip_special_tokens=True)
    burstiness = calculate_burstiness(generated_text)
    print(f"Burstiness: {burstiness}")

    # Save the fine-tuned model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [ ]:
text_file='/content/drive/MyDrive/FoodBuddy/unsupervised_train.txt'
fine_tune_gpt2("mbien/recipenlg",text_file,test_file, output_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt
# Load loss values from file
losses = np.load('losses.npy')

# Load learning rate values from file
lrs = np.load('lrs.npy')

# Plot loss as a function of learning rate
plt.plot(lrs, losses)
plt.xlabel('Learning Rate')
plt.ylabel('Loss')
plt.title('Learning Rate Finder')
plt.xscale('log')
plt.ylim(0, max(losses))
plt.grid(True)
plt.show()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Assuming you have specified your fine-tuned model directory
model_dir = output_dir

# Load tokenizer and model

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

# Generate text using the fine-tuned model
generated_text = model.generate(
    max_length=100,  # Adjust the length of generated text
    temperature=0.7,
    top_k=50,
    do_sample=True,
    num_return_sequences=1
)

generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(generated_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


IPE_ START>  chocolate chips  butterscotch chips  condensed milk  pecans  coconut  vanilla   1 small pkg. chocolate chips  1 small pkg. butterscotch chips  1 can sweetened condensed milk  1 c. chopped pecans  1 c. coconut  1 tsp. vanilla   Melt chips and milk together.  Add pecans


In [ ]:
# from flask import Flask, render_template, request, jsonify

In [ ]:
# ! pip install pyngrok

In [ ]:
# !pip install Flask Jinja2

In [ ]:
# !pip install flask==0.12.2
# !pip install flask-ngrok
# # print flask version
# from flask import Flask
# from flask_ngrok import run_with_ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: flask
    Found existing installation: Flask 2.2.5
    Uninstalling Flask-2.2.5:
      Successfully uninstalled Flask-2.2.5


In [ ]:
# !ngrok authtoken '2Yobuvo9E68KBLZ9ChOd3WckM4b_7n6F8JuKQRXW7GbGAkj2B'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# import threading
# from flask import Flask, render_template, request, jsonify

In [ ]:
# !pip install Flask requests

In [ ]:
# app = Flask(__name__)
# run_with_ngrok(app)

# @app.route('/')
# def index():
#     # Render the main interface where users can input their cravings or ingredients.
#     return render_template('/content/drive/MyDrive/FoodBuddy/index.html')
# @app.route('/get_recipe', methods=['POST'])
# def get_recipe():
#     # Load tokenizer and model
#     tokenizer = AutoTokenizer.from_pretrained(model_dir)
#     model = AutoModelForCausalLM.from_pretrained(model_dir)
#     try:
#         user_input = request.json['user_input']
#         # Tokenize the user input
#         input_ids = tokenizer.encode(user_input, return_tensors='pt')
#         # Generate a response from the model
#         gpt_response = model.generate(
#             input_ids,
#             max_length=100,
#             num_beams=5,
#             no_repeat_ngram_size=2,
#             top_k=10,
#             top_p=0.95,
#             temperature=0.7
#         )
#         # Decode the generated response
#         decoded_response = tokenizer.decode(gpt_response[0], skip_special_tokens=True)
#         return jsonify({'response': decoded_response, 'status': 'success'})
#     except Exception as e:
#         # In case of an error, return an error message
#         return jsonify({'response': str(e), 'status': 'error'})

# app.run()